In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
credits= pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv')
movies = pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv')

In [ ]:
credits.head()

In [ ]:
movies.head()

In [ ]:
movies.isnull().sum()

In [ ]:
movies.shape

In [ ]:
credits.shape

In [ ]:
credits.isnull().sum()

In [ ]:
movies_data = movies.merge(credits,left_on='id', right_on='movie_id')

In [ ]:
movies_data.head()

In [ ]:
movies_data.columns

# we just have to keep those columns which are related to tagging the movies for recommendation to user

In [ ]:
print(len(movies_data['title_x']))
print(len(movies_data['title_y']))

In [ ]:
movies_data = movies_data[['id','title_x','genres','keywords','overview','cast', 'crew']]
movies_data.head()

In [ ]:
movies_data.isnull().sum()

In [ ]:
movies_data = movies_data.dropna()
movies_data.isnull().sum()

In [ ]:
movies_data.duplicated().sum()

In [ ]:
movies_data.iloc[0].genres

In [ ]:
import ast

def convert(obj):
    l = []
    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l

In [ ]:
movies_data['genres'] = movies_data['genres'].apply(convert)

In [ ]:
movies_data['keywords'] = movies_data['keywords'].apply(convert)

In [ ]:
movies_data.head()

In [ ]:
movies_data.iloc[0].cast

In [ ]:
def convert_3(obj):
    l = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            l.append(i['name'])
            counter += 1
        else:
            break
    return l

In [ ]:
movies_data['cast'] = movies_data['cast'].apply(convert_3)

In [ ]:
movies_data.iloc[0].crew

In [ ]:
def fetch_director(obj):
    l = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            l.append(i['name'])
            break
    return l

In [ ]:
movies_data['crew'] = movies_data['crew'].apply(fetch_director)

In [ ]:
movies_data.head()

In [ ]:
movies_data['overview'] = movies_data['overview'].apply(lambda x: x.split())

In [ ]:
movies_data.head()

In [ ]:
movies_data = movies_data.rename(columns={'title_x':'title'})

In [ ]:
movies_data

In [ ]:
cols = ['genres','keywords','cast','crew'] 
for c in cols:
    movies_data[c] = movies_data[c].apply(lambda x: [i.replace(' ','') for i in x]) 

In [ ]:
movies_data.head()

In [ ]:
movies_data['tags'] = movies_data['overview'] + movies_data['genres'] + movies_data['keywords'] + movies_data['cast'] + movies_data['crew']
movies_data.head()

In [ ]:
new_df = movies_data[['id','title','tags']]
new_df.head()

In [ ]:
new_df = new_df.copy()
new_df['tags'] = new_df['tags'].apply(lambda x: ' '.join(x))

In [ ]:
new_df.head()

In [ ]:
new_df['tags'][0]

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())
new_df.head()

# Text vectorization - Bag of Words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
vectors = cv.fit_transform(new_df['tags']).toarray()
vectors

In [ ]:
vectors[0]

In [ ]:
print(cv.get_feature_names_out().tolist())

# stemming
loved love loving loves --> love

In [ ]:
import nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
    l = []
    for i in text.split():
        l.append(ps.stem(i))
    return " ".join(l)
    

In [ ]:
new_df['tags'] = new_df['tags'].apply(stem)

In [ ]:
vectors = cv.fit_transform(new_df['tags']).toarray()
vectors

In [ ]:
vectors[0]

In [ ]:
print(cv.get_feature_names_out().tolist())

# We have 4806 movies and 4806 vectors of it and 5000 words for each vector
# we will find the cosine distance ie the angle between vectors
# Euclidean distance is not a reliable measure in Higher dimensions

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(vectors)

# movie to movie similarity score

In [ ]:
similarity[0]

In [ ]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse=True, key= lambda x:x[1])[1:6]

    for i in movies_list:
        print(new_df.iloc[i[0]].title)
        

In [ ]:
recommend('Batman Begins')